## RNN demo

<br />
<br />
November 12, 2024

### Data loading

In [1]:
"""
Minimal character-level Vanilla RNN model. Written by Andrej Karpathy (@karpathy)
BSD License
"""
import numpy as np

# data I/O
data = open('resources/input.txt', 'r', encoding='utf-8').read() # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = { ch:i for i,ch in enumerate(chars) }
ix_to_char = { i:ch for i,ch in enumerate(chars) }

data has 160228 characters, 173 unique.


In [2]:
print(data[:590])

St. Petersburg mathematicians and 
their discoveries 
Editors: Nikita Kalinin, Stanislav Smirnov, Nikolai Reshetikhin, Galina Sinkevich, Dmitriy Stolyarov, Anastasia Reshetikhin 
Moscow 
MMXXIV

St. Petersburg mathematicians and their discoveries 
Editors: Nikita Kalinin, Stanislav Smirnov, Nikolai Reshetikhin, Galina Sinkevich, Dmitriy Stolyarov, Anastasia Reshetikhin Translators: Nodar Lakhuti, Lukeria Petrunina 
First page picture: Elizaveta Plis 
Book cover, preparing illustrations for printing: Alexander Ivanov Layout and typesetting: Olga Shirokova 
The concept of the book: Sta


In [3]:
' '.join(chars)

'\uf8f3 < c – а i ? Æ ] φ ω I г @ G … ´ Р H × } ” • 0 К U ∞ . Y Π S ∂ % W | d µ F a ч 3 \x81 s \uf8f4 у ˆ ∩ ` l ) τ > ≥ − \n / 4 ∧ 8 ; ĳ g Л к q ν ı 6 v 1 n w O { ∆ ¨ h © \x8c ± V Г z \uf8f1 χ 5 J Q \x82 M р ( + A P ’ з , х ⊥ œ o ¸ [ · Ω E н j u L r t м y \uf8f2 ≤ \\ и B \x8b б = ь ц * 7 ⊂ X N ы о ¦ k b Γ л T : ρ Z ∈ С ˇ & D Т О x η e — ! ψ 2 K “ ∇   с - p т f C R е д σ λ m → Œ'

### Initialization

In [4]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

<div align="left">
    <img src="images/rnn_char_level_example.jpg" width=400/>
</div>

### Loss function

 * forward pass for loss calculation
 * backpropagation of error

Let us denote the components of the probability vector of symbols (classes) at the RNN output as
$$ p_k = \frac{e^{f_k}}{\sum\limits_j e^{f_j}}$$

Then the value of the loss function on the next object $x_i$
$$ L_i = -\ln (p_{y_i})$$

Then, by direct calculation of the derivative, we can obtain
$$ \frac{\partial L_i}{\partial f_k} = p_k - 1 \cdot [y_i = k]$$

In [5]:
def lossFun(inputs, targets, hprev):
    """
    inputs, targets are both list of integers
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size, 1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [6]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

### The main loop

In [7]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [8]:
while n < 10**5 + 1000: #True 
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]
  
    # sample from the model now and then
    if n % 1000 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))
  
    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = lossFun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 1000 == 0: 
        print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
    
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                  [dWxh, dWhh, dWhy, dbh, dby], 
                                  [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
  
    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 з5AÆГxз7σKK—´[{*ĳ∂AhK4См“Wх? ’+КBzaUZ∈к¸ь∩vрГ`Π8<ˆq≤v:∇:&\W/zи:qtКjСχ0а¨¦µS`∞з
:д≥©kµ*ρ¸∞аткEρкP>}PL—ω[`bд/œ=∆O≥∞g´≤PŒΩVe,7a<Γ−−pψеφτ∇·≥]GхLРœV\œ+т4dtцNAгчwу1v@еКмτ8%gYµı-ĳ-7v¸w∈“Т,о±frк6DUL.[ 
----
iter 0, loss: 128.832299
----
 metrrenweitatetinveetyosthatt Botiosns fusen the hhenstecithes aag’eficy at of t. 
ities afired flemet wdvantthinddde thy acecuiveteteesd on anthe Sente,eties af, hes fangerrrrwibids Stemind the bduls 
----
iter 1000, loss: 91.800480
----
 (y tareds for unce pgergtŒTablicug thss has talil tif unlite an yimyin Jhis an. Ik wonnat ol Banl the femsevpindeve “eonoocgied amantari: ar an the thetild c. 
Dandetto serle icimaery w se ta hisseogi 
----
iter 2000, loss: 74.373669
----
 it in ardhemitymaterne buber =of. The  enuntoravid iss, n preco mots ssy s. E, “) Furn: ce.  uld s wablacal ppend, Em and ut cod andes hag dary the a der of. Bir mion tresitheredy, cateins en. gileuso 
----
iter 3000, loss: 66.391509
----
 artinige, Puted St ecind Fl´id bur t

### Till $10^5$ iterations

----

 uan, the engrix neind reeof-lame for the pinersgines chicles the cimber a e todemie ne amon Acadea’s this ampert: Allec interklow. Buwtion that the phere of uhe its on’s resodecemuttion: mumbliss in S 

----
iter 100000, loss: 46.196448
 